<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
# import piplite
# await piplite.install(['folium'])
# await piplite.install(['pandas'])

import folium


In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [4]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
import pandas as pd
import requests
from io import StringIO

# Download CSV directly
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(url)
spacex_df = pd.read_csv(StringIO(response.text))

# Verify
print(spacex_df.head())

   Flight Number        Date Time (UTC) Booster Version  Launch Site  \
0              1  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1              2  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2              3  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3              4  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4              5  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  Payload Mass (kg)  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats,  barrel o...                0.0   
2                             Dragon demo flight C2+              525.0   
3                                       SpaceX CRS-1              500.0   
4                                       SpaceX CRS-2              677.0   

       Orbit         Customer        Landing Outcome  class        Lat  \
0        LEO           SpaceX  Failure   (

In [6]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [7]:
spacex_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flight Number      56 non-null     int64  
 1   Date               56 non-null     object 
 2   Time (UTC)         56 non-null     object 
 3   Booster Version    56 non-null     object 
 4   Launch Site        56 non-null     object 
 5   Payload            56 non-null     object 
 6   Payload Mass (kg)  56 non-null     float64
 7   Orbit              56 non-null     object 
 8   Customer           56 non-null     object 
 9   Landing Outcome    56 non-null     object 
 10  class              56 non-null     int64  
 11  Lat                56 non-null     float64
 12  Long               56 non-null     float64
dtypes: float64(3), int64(2), object(8)
memory usage: 5.8+ KB


Now, you can take a look at what are the coordinates for each site.


In [8]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [9]:
launch_sites_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Launch Site  4 non-null      object 
 1   Lat          4 non-null      float64
 2   Long         4 non-null      float64
dtypes: float64(2), object(1)
memory usage: 224.0+ bytes


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [10]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [11]:
print(site_map)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [12]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [13]:
launch_sites_df.info()
launch_sites_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Launch Site  4 non-null      object 
 1   Lat          4 non-null      float64
 2   Long         4 non-null      float64
dtypes: float64(2), object(1)
memory usage: 224.0+ bytes


,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [14]:
# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=8)

for _, row in launch_sites_df.iterrows():
    # Add circle with popup (optional)
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=5000,
        color='orange',
        fill=True,
        popup=row['Launch Site']  # This creates a clickable popup (optional)
    ).add_to(site_map)

        # Customize label position based on site name
    if "CCAFS LC-40" in row["Launch Site"]:
        icon_anchor = (25, 0)   # Shift label to the right
    elif "CCAFS SLC-40" in row["Launch Site"]:
        icon_anchor = (-25, 0)  # Shift label to the left
    elif "KSC LC-39A" in row["Launch Site"]:
        icon_anchor = (0, 25)   # Shift label upward
    else:
        icon_anchor = (0, -25)  # Shift label downward

    # Add text label next to the circle
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.DivIcon(
            icon_size=(150, 30),
            icon_anchor=icon_anchor,
            html=f'<div style="font-size: 8pt; color: black; font-weight: bold;">{row["Launch Site"]}</div>'
        )
    ).add_to(site_map)

site_map  # Display the map (works in Jupyter)

In [15]:
import folium
import pandas as pd


# Initialize the map (use mean coordinates for centering)
nasa_coordinate = [launch_sites_df["Lat"].mean(), launch_sites_df["Long"].mean()]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add circles with popup labels
for _, row in launch_sites_df.iterrows():
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,  # Adjust radius as needed (e.g., 5000 for larger circles)
        color="blue",
        fill=True,
        fill_opacity=0.2,  # Optional: adjust transparency
        popup=row['Launch Site']  # Corrected parameter name
    ).add_to(site_map)




    folium.Marker(
        location=[row['Lat'], row['Long']],  # Same coordinates as the circle
        icon=folium.DivIcon(
            icon_size=(150, 30),  # Adjust based on text length
            icon_anchor=(25, 0),  # Shift label to the right of the circle
            html=f'<div style="font-size: 8pt; color: black; font-weight: bold;">{row["Launch Site"]}</div>'
        )
    ).add_to(site_map)

# ===== DISPLAY THE MAP =====
# Option 1: Display in Jupyter Notebook
site_map

# Option 2: Save to HTML and open in browser
# site_map.save("spacex_launch_sites.html")
# Then open the .html file manually or use:
# import webbrowser
# webbrowser.open("spacex_launch_sites.html")

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [16]:
# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [17]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [18]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [19]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

def create_marker_color(df):
    colors = []
    for i, row in df.iterrows():
        if row.iloc[3] == 1:
            # print('green')
            colors.append( 'green')
        elif row.iloc[3] == 0:
            # print('red')
            colors.append( 'red')
        else:
            # print(None)
            colors.append(None)

    df['marker_color'] = colors

create_marker_color(df=spacex_df)

print(spacex_df.info())
print(spacex_df.head())
print(spacex_df['marker_color'].value_counts())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Launch Site   56 non-null     object 
 1   Lat           56 non-null     float64
 2   Long          56 non-null     float64
 3   class         56 non-null     int64  
 4   marker_color  56 non-null     object 
dtypes: float64(2), int64(1), object(2)
memory usage: 2.3+ KB
None
   Launch Site        Lat       Long  class marker_color
0  CCAFS LC-40  28.562302 -80.577356      0          red
1  CCAFS LC-40  28.562302 -80.577356      0          red
2  CCAFS LC-40  28.562302 -80.577356      0          red
3  CCAFS LC-40  28.562302 -80.577356      0          red
4  CCAFS LC-40  28.562302 -80.577356      0          red
red      32
green    24
Name: marker_color, dtype: int64


In [20]:
# other approaches
import numpy as np
spacex_df_2 = spacex_df.copy()

spacex_df_2['marker_color'] = np.where(spacex_df_2['class']==1, 'green', 'red')

spacex_df_2.head()
spacex_df_2['marker_color'].value_counts()

red      32
green    24
Name: marker_color, dtype: int64

In [21]:
spacex_df_2['marker_color'] = spacex_df_2['class'].map({1:'green', 0: 'red'})

spacex_df_2['marker_color'].value_counts()

red      32
green    24
Name: marker_color, dtype: int64

In [22]:
spacex_df_2['marker_color'] = \
spacex_df_2['class'].apply(lambda x: 'orange' if x == 1 else ('lime' if x == 0 else 'crimson'))
spacex_df_2['marker_color'].value_counts()

lime      32
orange    24
Name: marker_color, dtype: int64

In [23]:
spacex_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Launch Site   56 non-null     object 
 1   Lat           56 non-null     float64
 2   Long          56 non-null     float64
 3   class         56 non-null     int64  
 4   marker_color  56 non-null     object 
dtypes: float64(2), int64(1), object(2)
memory usage: 2.3+ KB


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [24]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    # lats[k], lons[k]

    location = record['Lat'], record['Long']
    marker = folium.Marker(location=location)
    icon=folium.Icon(color='white', icon_color=record['marker_color'])
    popup = 'lon:{}<br>lat:{}'.format(location[1], location[0])
    folium.Popup(popup).add_to(marker)
    marker_cluster.add_child(marker)
    # marker_cluster.add_child(marker)

site_map

In [25]:
# Add marker_cluster to the map (correct)
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    # Get coordinates
    location = [record['Lat'], record['Long']]
    
    # Create marker with customized icon and popup
    marker = folium.Marker(
        location=location,
        popup=f"lon:{location[1]}<br>lat:{location[0]}",  # Modern f-string
        icon=folium.Icon(
            color='white',  # Base color
            icon_color=record['marker_color']  # Color from your dataframe
        )
    )
    # Add to cluster (once)
    marker.add_to(marker_cluster)

# No need to add marker_cluster again here

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [26]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [27]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [28]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [29]:
coastline_point = [28.56367, -80.57163]  # Example coastline coordinates
launch_site = [28.56321, -80.57682]      # Example launch site coordinates
distance_km = 5.42      

calculate_distance(28.56367, -80.57163, 28.56321, -80.57682)

0.5095958694058215

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [30]:
# Launch site: lon:-80.64689529 lat:28.57325457
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [31]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

from folium.features import DivIcon

# Assuming you have these coordinates:
coastline_point = [28.56367, -80.57163]  # Example coastline coordinates
launch_site = [28.56321, -80.57682]      # Example launch site coordinates
distance_km = 5.42                        # Calculated distance between points

# Create the distance marker
distance_marker = folium.Marker(
    location=coastline_point,
    icon=DivIcon(
        icon_size=(150, 30),  # Width, height in pixels
        icon_anchor=(0, 0),   # Position of icon relative to location
        html=f"""
        <div style="
            font-size: 12pt; 
            color: #d35400;
            background: white;
            padding: 3px;
            border-radius: 3px;
            border: 1px solid #d35400;
        ">
            <b>{distance_km:.2f} KM</b>
        </div>
        """
    )
)

# Add to your existing map
distance_marker.add_to(site_map)

# Display the map (if in Jupyter notebook)
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [32]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
from folium.vector_layers import PolyLine

# Coordinates (example - replace with your actual coordinates)
coastline_coord = [28.56367, -80.57163]  # Coastline point
launch_site_coord = [28.56321, -80.57682]  # Launch site
distance_km = 5.42  # Pre-calculated distance

# 1. Create the PolyLine
lines = PolyLine(
    locations=[coastline_coord, launch_site_coord],
    weight=2,  # Line thickness
    color='#d35400',  # Orange color
    dash_array='5, 5'  # Dashed line (remove for solid line)
).add_to(site_map)

# 2. Add distance label at midpoint
midpoint = [
    (coastline_coord[0] + launch_site_coord[0])/2,
    (coastline_coord[1] + launch_site_coord[1])/2
]

folium.Marker(
    midpoint,
    icon=DivIcon(
        icon_size=(150,30),
        icon_anchor=(75,15),  # Centers the label
        html=f"""
        <div style="
            font-size: 12pt;
            color: #d35400;
            background: white;
            border: 1px solid #d35400;
            border-radius: 3px;
            text-align: center;
        ">
            <b>{distance_km:.2f} KM</b>
        </div>
        """
    )
).add_to(site_map)

# 3. Optional: Add endpoints with different markers
folium.Marker(
    coastline_coord,
    popup="Coastline Point",
    icon=folium.Icon(color='blue', icon='flag')
).add_to(site_map)

folium.Marker(
    launch_site_coord,
    popup="Launch Site",
    icon=folium.Icon(color='red', icon='rocket')
).add_to(site_map)

# Refresh display (in Jupyter)

site_map.add_child(lines)

site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [33]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [34]:
from folium.vector_layers import PolyLine
from folium.features import DivIcon
import folium

# Titusville coordinates (example)
titusville_coord = [28.6122, -80.8076]  # Titusville, FL
launch_site_coord = [28.56321, -80.57682]  # Example launch site (Cape Canaveral)
distance_km = 20.5  # Example distance (calculate real distance with geopy)

# 1. Create the PolyLine connection
lines = PolyLine(
    locations=[titusville_coord, launch_site_coord],
    weight=2,
    color='#d35400',
    dash_array='5, 5'
).add_to(site_map)

# 2. Add distance label at midpoint
midpoint = [
    (titusville_coord[0] + launch_site_coord[0])/2,
    (titusville_coord[1] + launch_site_coord[1])/2
]

folium.Marker(
    midpoint,
    icon=DivIcon(
        icon_size=(150,30),
        icon_anchor=(75,15),
        html=f"""
        <div style="
            font-size: 12pt;
            color: #d35400;
            background: white;
            border: 1px solid #d35400;
            border-radius: 3px;
            text-align: center;
        ">
            <b>{distance_km:.2f} KM to Titusville</b>
        </div>
        """
    )
).add_to(site_map)

# 3. Add special marker for Titusville
folium.Marker(
    titusville_coord,
    popup="Titusville, FL",
    icon=folium.Icon(color='green', icon='city', prefix='fa')  # Using Font Awesome city icon
).add_to(site_map)

# 4. Keep existing launch site marker
folium.Marker(
    launch_site_coord,
    popup="Launch Site",
    icon=folium.Icon(color='red', icon='rocket')
).add_to(site_map)

# Automatically zoom to show both locations
site_map.fit_bounds([titusville_coord, launch_site_coord])

# Display the map
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


In [35]:
pip show folium

Note: you may need to restart the kernel to use updated packages.Name: folium
Version: 0.19.6
Summary: Make beautiful maps with Leaflet.js & Python
Home-page: https://github.com/python-visualization/folium
Author: Rob Story
Author-email: wrobstory@gmail.com
License: MIT
Location: c:\users\gamarandor\miniconda3\envs\plotly_dash\lib\site-packages
Requires: jinja2, numpy, branca, requests, xyzservices
Required-by: 



## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
